<a href="https://colab.research.google.com/github/bueorm/ALM/blob/main/ALM_new_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
from pickle import load
from google.colab import drive
np.random.seed(0)

In [3]:
filename = '/content/drive/MyDrive/ALM MODEL/data-pretrain-alp003.pkl'

dataset = load(open(filename, 'rb'))
print(dataset[0]['input'])
print(dataset[0]['output'])

OpenAI es una empresa de investigación y despliegue de inteligencia artificial que declara tener como misión asegurar que la inteligencia artificial general beneficie a toda la humanidad.4​

Algunas de sus áreas de enfoque son los modelos generativos, la seguridad, la robótica y el lenguaje natural. OpenAI también ofrece una plataforma de API que permite a los usuarios acceder y construir sus propios modelos de IA personalizados.

OpenAI fue fundada en 2015 por Ilya Sutskever, Greg 
OpenAI es una empresa de investigación y despliegue de inteligencia artificial que declara tener como misión asegurar que la inteligencia artificial general beneficie a toda la humanidad.4​

Algunas de sus áreas de enfoque son los modelos generativos, la seguridad, la robótica y el lenguaje natural. OpenAI también ofrece una plataforma de API que permite a los usuarios acceder y construir sus propios modelos de IA personalizados.

OpenAI fue fundada en 2015 por Ilya Sutskever, Greg Brockman, Trevor Blackwel

In [4]:
# Crear "tokens"
source_tokens = []
for sentence in dataset:
    data = sentence['input']
    source_tokens.append(data.split(' '))
print(source_tokens[1])

target_tokens = []
for sentence in dataset:
    data = sentence['output']
    target_tokens.append(data.split(' '))
print(target_tokens[1])

['OpenAI', 'se', 'fundó', 'en', '2015', 'como', 'una', 'organización', 'sin', 'fines', 'de', 'lucro,', 'con', 'la', 'misión', 'de', 'asegurar', 'que', 'la', 'inteligencia', 'artificial', 'general', '(IAG)', 'beneficie', 'a', 'toda', 'la', 'humanidad.', 'Entre', 'sus', 'fundadores', 'se', 'encontraban', 'Elon', 'Musk,', 'Sam', 'Altman,', 'Ilya', 'Sutskever,', 'Greg', 'Brockman', 'y', 'otros', 'expertos', 'en', 'IA.\n\nOpenAI', 'lanzó', 'Universe,', 'una', 'plataforma', '']
['OpenAI', 'se', 'fundó', 'en', '2015', 'como', 'una', 'organización', 'sin', 'fines', 'de', 'lucro,', 'con', 'la', 'misión', 'de', 'asegurar', 'que', 'la', 'inteligencia', 'artificial', 'general', '(IAG)', 'beneficie', 'a', 'toda', 'la', 'humanidad.', 'Entre', 'sus', 'fundadores', 'se', 'encontraban', 'Elon', 'Musk,', 'Sam', 'Altman,', 'Ilya', 'Sutskever,', 'Greg', 'Brockman', 'y', 'otros', 'expertos', 'en', 'IA.\n\nOpenAI', 'lanzó', 'Universe,', 'una', 'plataforma', 'de', 'software', 'para', 'medir', 'y', 'entrenar'

In [5]:
def build_token_dict(token_list):
  token_dict = {
      '<PAD>': 0,
      '<START>': 1,
      '<END>': 2
  }
  for tokens in token_list:
    for token in tokens:
      if token not in token_dict:
        token_dict[token] = len(token_dict)
  return token_dict

In [6]:
source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v:k for k,v in target_token_dict.items()}

print(source_token_dict)
print(target_token_dict)
print(target_token_dict_inv)

{'<PAD>': 0, '<START>': 1, '<END>': 2, 'OpenAI': 3, 'es': 4, 'una': 5, 'empresa': 6, 'de': 7, 'investigación': 8, 'y': 9, 'despliegue': 10, 'inteligencia': 11, 'artificial': 12, 'que': 13, 'declara': 14, 'tener': 15, 'como': 16, 'misión': 17, 'asegurar': 18, 'la': 19, 'general': 20, 'beneficie': 21, 'a': 22, 'toda': 23, 'humanidad.4\u200b\n\nAlgunas': 24, 'sus': 25, 'áreas': 26, 'enfoque': 27, 'son': 28, 'los': 29, 'modelos': 30, 'generativos,': 31, 'seguridad,': 32, 'robótica': 33, 'el': 34, 'lenguaje': 35, 'natural.': 36, 'también': 37, 'ofrece': 38, 'plataforma': 39, 'API': 40, 'permite': 41, 'usuarios': 42, 'acceder': 43, 'construir': 44, 'propios': 45, 'IA': 46, 'personalizados.\n\nOpenAI': 47, 'fue': 48, 'fundada': 49, 'en': 50, '2015': 51, 'por': 52, 'Ilya': 53, 'Sutskever,': 54, 'Greg': 55, '': 56, 'se': 57, 'fundó': 58, 'organización': 59, 'sin': 60, 'fines': 61, 'lucro,': 62, 'con': 63, '(IAG)': 64, 'humanidad.': 65, 'Entre': 66, 'fundadores': 67, 'encontraban': 68, 'Elon': 6

In [7]:
# Agregar start, end y pad a cada frase del set de entrenamiento
encoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>'] for tokens in target_tokens]

source_max_len = max(map(len, encoder_tokens))
target_max_len = max(map(len, decoder_tokens))

encoder_tokens = [tokens + ['<PAD>']*(source_max_len-len(tokens)) for tokens in encoder_tokens]
decoder_tokens = [tokens + ['<PAD>']*(target_max_len-len(tokens)) for tokens in decoder_tokens]
output_tokens = [tokens + ['<PAD>']*(target_max_len-len(tokens)) for tokens in output_tokens ]

In [8]:
encoder_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encoder_tokens]
decoder_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decoder_tokens]
output_decoded = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Asegurar que la dimensión del modelo (d_model) sea divisible por el número de cabezas
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Inicilizar Dimensiones
        self.d_model = d_model # Dimension del modelo
        self.num_heads = num_heads # numeros de cabezas
        self.d_k = d_model // num_heads # Dimensión de la clave, consulta y valor de cada cabeza

        # Capas lineales para transformar entradas
        self.W_q = nn.Linear(d_model, d_model) #Transformación de consulta
        self.W_k = nn.Linear(d_model, d_model) #Transformación de clave
        self.W_v = nn.Linear(d_model, d_model) #Transformación de valor
        self.W_o = nn.Linear(d_model, d_model) #Transformación de salida

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        #Calcular puntuaciones de atención
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Aplicar máscara si se proporciona (útil para evitar la atención a ciertas partes como el relleno)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        #Se aplica Softmax para obtener probabilidades de atención
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiplicar por valores para obtener la salida final
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reformar la entrada para tener num_heads para atención multi-cabeza
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        # Combinar las múltiples cabezas de nuevo a la forma original
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        #Aplicar transformaciones lineales y dividir cabezas
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Realizar atención de producto escalar punteado
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combinar cabezas y aplicar transformación de salida
        output = self.W_o(self.combine_heads(attn_output))
        return output

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x


In [10]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [11]:
src_vocab_size = max(len(source_token_dict),len(target_token_dict))
tgt_vocab_size = max(len(source_token_dict),len(target_token_dict))
d_model = 512
num_heads = 8
num_layers = 8
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [12]:
x = [np.array(encoder_input), np.array(decoder_input)]
y = np.array(output_decoded)

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam

# Assuming you have your Transformer model defined as 'transformer'

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(30):
  optimizer.zero_grad()

  # Access encoded data from your pre-processing step
  src_data = x[0]  # Assuming x[0] contains encoded source data
  tgt_data = y  # Assuming y contains encoded target data

  # Convert NumPy arrays to PyTorch tensors
  src_data = torch.from_numpy(src_data)
  tgt_data = torch.from_numpy(tgt_data)

  # Eliminate the last token from the target sequence (for prediction)
  tgt_data = tgt_data[:, :-1]

  # Feed encoded data to the model
  output = transformer(src_data, tgt_data)

  # Calculate loss
  loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))

  # Update parameters
  loss.backward()
  optimizer.step()

  print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total number of parameters: {total_params}")

torch.save(transformer, "mi_modelo_transformer.pt")


In [ ]:
torch.save(transformer, "mi_modelo_transformer.pt")
#loaded_model = torch.load("mi_modelo_transformer.pt")

In [ ]:
def predict(sentence, model, source_token_dict, target_token_dict_inv, start_token, end_token, pad_token):
  # Preprocesar la entrada
  sentence_tokens = sentence.split(' ')
  sentence_tokens = [sentence_tokens + ['<END>', '<PAD>']]
  tr_input = torch.from_numpy(np.array([source_token_dict[token] for token in sentence_tokens[0]])).unsqueeze(0)

  # Cambiar el modelo a modo evaluación
  model.eval()

  # Decodificar la salida del modelo
  with torch.no_grad():
      output = model(tr_input)
      decoded = output.argmax(dim=-1).squeeze()  # Obtener los índices de los tokens predichos

  # Post-procesar la salida
  predicted_sentence = [target_token_dict_inv[token.item()] for token in decoded[1:-1] if token.item() not in [start_token, end_token, pad_token]]  # Filtrar tokens especiales
  print('INPUT: {}'.format(sentence))
  print('.....................................................................................................')
  print('OUTPUT: {}'.format(' '.join(predicted_sentence)))

In [ ]:
# Ejemplo de uso
your_sentence = "Hola, ¿cómo estás?"
predict(your_sentence, loaded_model, source_token_dict, target_token_dict_inv, target_token_dict['<START>'], target_token_dict['<END>'], target_token_dict['<PAD>'])